# Construction du modèle de reconnaissance de caractère basé sur le dataset MIST

Dans ce notebook sera présenté la création du modèle qui permettra de **classifier** les lettres de l'alphabet de a-z et A-Z.

Pour ce faire nous allons utiliser le dataset d'images prétraités créés par le notebook [NIST-preprocessing](/notebooks/notebooks/character_recognition/NIST-preprocessing.ipynb)

Pour exécuter ce notebook, veillez à ce que le jeu de données soit bien sous la forme suivante.
- data
    - processed
         - NIST-dataset
            - train
                - a000001.png
                - a000002.png
                ...
                ...
                - a00000n.png
            - test_set
                - a000001.png
                - a000002.png
                ...
                ...
                - a00000n.png


## Importation des dépendances

In [36]:
import os
from random import shuffle
from string import ascii_lowercase, ascii_uppercase

import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from tensorflow import keras
from tensorflow.keras import layers

In [37]:
import tensorflow as tf
from tensorflow.python.client import device_lib

print("Num of GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
device_lib.list_local_devices()

Num of GPUs Available:  0


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 13510354965348025898
 xla_global_id: -1]

In [38]:
tf.test.is_built_with_cuda()
tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)

False

## Définition des paramètres du notebook

In [39]:
NIST_PROCESSED_PATH = "../../data/processed/NIST-dataset"
TRAIN_SET_PATH = os.path.join(NIST_PROCESSED_PATH, "train")
TEST_SET_PATH = os.path.join(NIST_PROCESSED_PATH, "test")

## Importation du dataset

Dans cette section, nous allons charger le dataset afin d'obtenir un train_data_set, train_label_set, ainsi qu'un test_data_set et un test_label_set

### Chargement du dataset depuis le disque

Déclaration de la fonction qui va charger l'image en récupérant le label associé à celle-ci

In [40]:
def load_img_to_np_array(file_path):
    full_name = os.path.basename(file_path)
    file_name = os.path.splitext(full_name)[0]
    label = file_name[0]

    return label, plt.imread(file_path)

Cette fonction vérifie si un fichier est bien un png.

In [41]:
def is_png(file_name):
    extension = file_name.split(".")[1]
    if extension == "png":
        return True
    return False

file_name1 = "AOO1.png"
file_name2 = "AOO1.jpeg"

print(f"{file_name1} est un png ? {is_png(file_name1)}")
print(f"{file_name2} est un png ? {is_png(file_name2)}")

AOO1.png est un png ? True
AOO1.jpeg est un png ? False


On récupère tous les fichiers .png dans un repertoire spécifié en paramètre (dir_path)


In [42]:
def select_png_from_dir(dir_path):
    files_in_dir = []
    with os.scandir(dir_path) as entries:
        for entry in entries:
            file_name = entry.name
            if entry.is_file() and is_png(file_name):
                file_path = os.path.join(dir_path, file_name)
                files_in_dir.append(file_path)
    return files_in_dir

On récupère tous les fichiers en extension png dans les répertoires train et test

In [43]:
train_img_list = select_png_from_dir(TRAIN_SET_PATH)
test_img_list = select_png_from_dir(TEST_SET_PATH)
train_img_list[0:5]

['../../data/processed/NIST-dataset\\train\\a0000.png',
 '../../data/processed/NIST-dataset\\train\\a0001.png',
 '../../data/processed/NIST-dataset\\train\\A0002.png',
 '../../data/processed/NIST-dataset\\train\\A0004.png',
 '../../data/processed/NIST-dataset\\train\\A0006.png']

Cette fonction a pour but de créer un data_set sous forme [ (label, image array), ... ]

In [44]:
def load_data_set(data_img_list):
    data_set = []

    for img_path in data_img_list:
        elem = load_img_to_np_array(img_path)
        data_set.append(elem)

    return data_set

On créer le set d'entrainement et de test

In [45]:
train_set = load_data_set(train_img_list)
test_set = load_data_set(test_img_list)
test_set[0]

('A',
 array([[0.99607843, 0.99607843, 0.99607843, ..., 0.99607843, 0.99607843,
         0.99607843],
        [0.99607843, 0.99607843, 0.99607843, ..., 0.99607843, 0.99607843,
         0.99607843],
        [0.99607843, 0.99607843, 0.99607843, ..., 0.99607843, 0.99607843,
         0.99607843],
        ...,
        [0.99607843, 0.99607843, 0.99607843, ..., 0.99607843, 0.99607843,
         0.99607843],
        [0.99607843, 0.99607843, 0.99607843, ..., 0.99607843, 0.99607843,
         0.99607843],
        [0.99607843, 0.99607843, 0.99607843, ..., 0.99607843, 0.99607843,
         0.99607843]], dtype=float32))

## Mélange des sets et codage des labels en chiffre

On mélange les sets

In [46]:
shuffle(train_set)
shuffle(test_set)

On récupère les données et leurs labels dans des listes distinctes

In [47]:
y_raw_train, x_raw_train = list(zip(*train_set))
y_raw_test, x_raw_test = list(zip(*test_set))
print(y_raw_train[0])
print(x_raw_train[0].shape)

A
(41, 41)


On récupère tous les labels possibles

In [48]:
label_list = [char for char in ascii_lowercase + ascii_uppercase]
print(label_list[0:5])

['a', 'b', 'c', 'd', 'e']


On définit l'encoder à utiliser

In [49]:
encoder = LabelBinarizer()
encoder.fit(label_list)

LabelBinarizer()

On encode nos données au format OneShot

In [50]:
y_train = encoder.transform(y_raw_train)
y_test = encoder.transform(y_raw_test)
y_train.shape

(233868, 52)

In [51]:
x_train = np.array(x_raw_train)
x_test = np.array(x_raw_test)
x_train.shape

(233868, 41, 41)

## Création du modèle de classification

In [58]:
model = keras.Sequential([
    keras.Input(shape=(41,41)),
    layers.Flatten(input_shape=(41, 41)),
    layers.Dense(100, activation="relu"),
    layers.Dense(100, activation="relu"),
    layers.Dense(100, activation="relu"),
    layers.Dense(100, activation="relu"),
    layers.Dense(52, activation="softmax"),
])

model.compile(optimizer="sgd", loss="categorical_crossentropy", metrics=["acc"])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_3 (Flatten)         (None, 1681)              0         
                                                                 
 dense_17 (Dense)            (None, 100)               168200    
                                                                 
 dense_18 (Dense)            (None, 100)               10100     
                                                                 
 dense_19 (Dense)            (None, 100)               10100     
                                                                 
 dense_20 (Dense)            (None, 100)               10100     
                                                                 
 dense_21 (Dense)            (None, 52)                5252      
                                                                 
Total params: 203,752
Trainable params: 203,752
Non-tr

## Entrainement du modèle

In [59]:
model.fit(x_train, y_train, epochs=20)

Epoch 1/20
7309/7309 [==============================] - 14s 2ms/step - loss: 2.0038 - acc: 0.4603
Epoch 2/20
7309/7309 [==============================] - 14s 2ms/step - loss: 1.2742 - acc: 0.6050
Epoch 3/20
7309/7309 [==============================] - 15s 2ms/step - loss: 1.0987 - acc: 0.6397
Epoch 4/20
7309/7309 [==============================] - 14s 2ms/step - loss: 1.0105 - acc: 0.6571
Epoch 5/20
7309/7309 [==============================] - 14s 2ms/step - loss: 0.9533 - acc: 0.6693
Epoch 6/20
7309/7309 [==============================] - 15s 2ms/step - loss: 0.9142 - acc: 0.6782
Epoch 7/20
7309/7309 [==============================] - 14s 2ms/step - loss: 0.8830 - acc: 0.6842
Epoch 8/20
7309/7309 [==============================] - 14s 2ms/step - loss: 0.8592 - acc: 0.6894
Epoch 9/20
7309/7309 [==============================] - 14s 2ms/step - loss: 0.8409 - acc: 0.6921
Epoch 10/20
7309/7309 [==============================] - 14s 2ms/step - loss: 0.8227 - acc: 0.6963
Epoch 11/20
7309/73

## Test du modèle sur le jeu de test

In [60]:
model.evaluate(x_test, y_test, verbose=2)

3546/3546 - 4s - loss: 0.8178 - acc: 0.6923 - 4s/epoch - 1ms/step


[0.8177857398986816, 0.6923456788063049]